In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
import random
import pymysql
import datetime

In [2]:
def get_data():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("SELECT flavor, menu_id, user_name FROM young_cheline.evaluate;")
    res = cursor.fetchall()
    return res

In [3]:
def make_ptable(df):
    ptable = df.pivot_table(values = 'flavor', index = 'menu_id', columns = 'user_name') 
    
    return ptable

In [4]:
def cosine(ptable):
    ptable = ptable.fillna(0)
    menu_id = ptable.index

    cos = cosine_similarity(ptable)
    cos = pd.DataFrame(data = cos, index=menu_id, columns=menu_id)
    return cos

In [5]:
def search_cos(cos,pt, user_id: str):
    ex = pt.assign(menu=pt.index.to_list())

    pt=ex.loc[ex[user_id].notnull(), [user_id, 'menu']] 
    user_cos=cos[pt['menu']] ## cos 확인 
    user_cos=user_cos[~user_cos.isin(user_cos.loc[pt['menu']])].dropna()
    X_vector = pt.iloc[:,[0]].values/10+1
    Y_matrix= user_cos.to_numpy()
    Matrix_product=Y_matrix.dot(X_vector)
    user_cos['Sum']=Matrix_product
    user_recomand=user_cos.sort_values('Sum',ascending=False).head(10).index.to_list()
    return user_recomand

In [6]:
def recommand(user_id):
    df=data_open()
    ptable = make_ptable(df)
    cos = cosine(ptable)
    a = search_cos(cos,ptable, user_id=user_id)
    print(user_id,a)
    return user_id, a

In [7]:
def data_open():
    data = get_data()
    df = pd.DataFrame(data, columns=['flavor', 'menu_id','user_name'])
    df['flavor'] = df['flavor'].astype(int)
    return df

In [8]:
df=data_open()

In [9]:
df

,flavor,menu_id,user_name
0,1,1,0045b6f3-8a
1,2,1,0047390f-82
2,2,1,004a7141-85
3,1,1,004ab9f1-86
4,2,1,005a45fe-34
...,...,...,...
99995,2,35,ffe679fb-1d
99996,1,35,ffe9b81b-04
99997,3,35,ffeaa9d2-e5
99998,4,35,ffeed759-bb


In [10]:
ptable = make_ptable(df)

In [11]:
ptable

user_name,000a0b61-26,001b5b79-4d,00294bd8-ec,003d84d4-41,0045b6f3-8a,0047390f-82,0049c40f-00,004a7141-85,004ab9f1-86,0053047d-b0,...,ffca521e-9f,ffcdd4d5-9e,ffd083d1-55,ffda803a-89,ffe679fb-1d,ffe7a6ab-60,ffe9b81b-04,ffeaa9d2-e5,ffeed759-bb,fff9b1b1-14
menu_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,1.0,2.0,NaN,2.0,1.0,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0
2,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,...,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,2.0,NaN,1.0,NaN,NaN
5,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,3.0,NaN,1.5,NaN,NaN
6,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,...,1.5,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN
8,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,1.5,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN
9,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN


In [12]:
cos = cosine(ptable)

In [13]:
cos

menu_id,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,30,31,32,33,34,35
menu_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.203030,0.233096,0.203860,0.204008,0.219642,0.205975,0.183610,0.203626,0.200952,...,0.213165,0.203905,0.193421,0.198053,0.208983,0.208600,0.199258,0.203285,0.211822,0.210456
2,0.203030,1.000000,0.221181,0.208214,0.204887,0.194516,0.204731,0.201057,0.212124,0.195558,...,0.212021,0.214050,0.201818,0.204685,0.209747,0.210535,0.224884,0.209842,0.205954,0.205393
3,0.233096,0.221181,1.000000,0.207698,0.214483,0.218404,0.215436,0.216695,0.205801,0.206835,...,0.212797,0.197619,0.223629,0.214182,0.221106,0.207078,0.213829,0.214693,0.211639,0.211012
4,0.203860,0.208214,0.207698,1.000000,0.202621,0.216352,0.218445,0.190436,0.210823,0.224418,...,0.191017,0.217690,0.197477,0.210492,0.195915,0.201153,0.207907,0.209846,0.206083,0.214772
5,0.204008,0.204887,0.214483,0.202621,1.000000,0.213329,0.213253,0.209130,0.213768,0.207441,...,0.220831,0.222392,0.217415,0.215330,0.220080,0.201670,0.210561,0.222115,0.203708,0.198461
6,0.219642,0.194516,0.218404,0.216352,0.213329,1.000000,0.203684,0.213713,0.190661,0.219984,...,0.213668,0.206944,0.202961,0.218467,0.220096,0.203722,0.197867,0.216306,0.197901,0.207533
7,0.205975,0.204731,0.215436,0.218445,0.213253,0.203684,1.000000,0.207996,0.208608,0.207183,...,0.208778,0.212425,0.213231,0.206950,0.223084,0.207386,0.200746,0.209683,0.215958,0.205420
8,0.183610,0.201057,0.216695,0.190436,0.209130,0.213713,0.207996,1.000000,0.192628,0.197987,...,0.224569,0.206661,0.198566,0.217006,0.205311,0.194082,0.192936,0.218808,0.199272,0.201896
9,0.203626,0.212124,0.205801,0.210823,0.213768,0.190661,0.208608,0.192628,1.000000,0.201290,...,0.219796,0.209651,0.209483,0.207485,0.205519,0.208059,0.210182,0.213164,0.187377,0.190092


In [14]:
a = search_cos(cos,ptable, user_id='ehddbs4521')

In [15]:
a

[33, 26, 30, 7, 21, 16, 29, 15, 14, 19]